In [3]:
from web3 import Web3
from solcx import compile_source

# Connect to the local Ganache blockchain
ganache_url = "http://127.0.0.1:7545"  
web3 = Web3(Web3.HTTPProvider(ganache_url))

# Check if the connection to the blockchain was successful
if not web3.is_connected():
    raise Exception("Unable to connect to Ganache, please make sure Ganache is running and listening on port 7545.")

# Use the account and private key provided by Ganache
private_key = "0xb0ac7e4efd83a4c1628466d5863e6ea98acbb69e7ec45a151ae2bfe311ec9807"  
wallet_address = web3.eth.account.from_key(private_key).address  # Generate wallet address from private key

# ERC20 contract source code
contract_source_code = '''
pragma solidity ^0.8.0;

contract MyToken {
    string public name = "TRIGGERCOIN";  // Token name
    string public symbol = "TGC";  // Token symbol
    uint8 public decimals = 18;  // Number of decimal places
    uint256 public totalSupply;  // Total supply

    mapping(address => uint256) public balanceOf;  // Mapping for user balances
    mapping(address => mapping(address => uint256)) public allowance;  // Mapping for allowances

    event Transfer(address indexed from, address indexed to, uint256 value);  // Transfer event
    event Approval(address indexed owner, address indexed spender, uint256 value);  // Approval event

    constructor(uint256 initialSupply) {  // Constructor to initialize total supply and creator's balance
        totalSupply = initialSupply * (10 ** uint256(decimals));  // Calculate actual supply
        balanceOf[msg.sender] = totalSupply;  // Creator gets all tokens
    }

    function transfer(address to, uint256 value) public returns (bool success) {  // Transfer function
        require(balanceOf[msg.sender] >= value, "Insufficient balance");  // Ensure sender has enough balance
        balanceOf[msg.sender] -= value;  // Decrease sender's balance
        balanceOf[to] += value;  // Increase recipient's balance
        emit Transfer(msg.sender, to, value);  // Trigger transfer event
        return true;
    }

    function approve(address spender, uint256 value) public returns (bool success) {  // Approve function
        allowance[msg.sender][spender] = value;  // Set allowance amount
        emit Approval(msg.sender, spender, value);  // Trigger approval event
        return true;
    }

    function transferFrom(address from, address to, uint256 value) public returns (bool success) {  // Transfer from function
        require(balanceOf[from] >= value, "Insufficient balance");  // Ensure sender has enough balance
        require(allowance[from][msg.sender] >= value, "Allowance exceeded");  // Ensure allowance is sufficient
        balanceOf[from] -= value;  // Decrease sender's balance
        balanceOf[to] += value;  // Increase recipient's balance
        allowance[from][msg.sender] -= value;  // Decrease allowance
        emit Transfer(from, to, value);  // Trigger transfer event
        return true;
    }
}
'''

# Compile the contract source code
compiled_sol = compile_source(contract_source_code, output_values=["abi", "bin"])
contract_id, contract_interface = compiled_sol.popitem()  # Get the compiled contract info
abi = contract_interface['abi']  # ABI: Application Binary Interface used to interact with the contract
bytecode = contract_interface['bin']  # Bytecode: The byte representation of the contract, used for deploying the contract

# Deploy the contract
contract = web3.eth.contract(abi=abi, bytecode=bytecode)
nonce = web3.eth.get_transaction_count(wallet_address)  # Get the current account's transaction count (nonce)

# Build the contract deployment transaction
transaction = contract.constructor(1000000).build_transaction({
    'chainId': 1337,  # Default Chain ID for Ganache
    'gas': 2000000,  # Estimated Gas limit
    'gasPrice': web3.to_wei('10', 'gwei'),  # Gas price
    'nonce': nonce,  # Transaction count
})

# Sign the transaction
signed_tx = web3.eth.account.sign_transaction(transaction, private_key)

# Send the signed transaction
tx_hash = web3.eth.send_raw_transaction(signed_tx.raw_transaction)

# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)

# Get the contract address
contract_address = tx_receipt.contractAddress
print(f"Contract deployed at address: {contract_address}")

# Interact with the deployed contract
deployed_contract = web3.eth.contract(address=contract_address, abi=abi)

# Transfer tokens
receiver_address = "0x4439862d0F33b6B84Fd3ad887f7214A24Aa43049"  # Replace with another account address provided by Ganache
transfer_tx = deployed_contract.functions.transfer(receiver_address, 1000).build_transaction({
    'chainId': 1337,    
    'gas': 2000000,
    'gasPrice': web3.to_wei('10', 'gwei'),
    'nonce': web3.eth.get_transaction_count(wallet_address),
    'from': wallet_address,  # Explicitly specify the sender address
})

# Sign the transfer transaction
signed_transfer_tx = web3.eth.account.sign_transaction(transfer_tx, private_key)

# Send the transfer transaction
transfer_tx_hash = web3.eth.send_raw_transaction(signed_transfer_tx.raw_transaction)

# Wait for the transfer transaction to complete, and get the transaction receipt
transfer_receipt = web3.eth.wait_for_transaction_receipt(transfer_tx_hash)

print(f"Transfer completed, transaction hash: {transfer_tx_hash.hex()}")

# Verify balances
sender_balance = deployed_contract.functions.balanceOf(wallet_address).call()
receiver_balance = deployed_contract.functions.balanceOf(receiver_address).call()
print(f"Sender balance: {sender_balance} TGC")
print(f"Receiver balance: {receiver_balance} TGC")


Contract deployed at address: 0x299652a43F3c70B45C5304B2E0A65fAe1EE1dcaE
Transfer completed, transaction hash: 976e953403a76b7f42d1ad253dfb5373dae206e91c4f722ac0abfbf28f66c0a5
Sender balance: 999999999999999999999000 TGC
Receiver balance: 1000 TGC


In [4]:
from web3 import Web3
from solcx import compile_source

# 连接到 Ganache 本地区块链
ganache_url = "http://127.0.0.1:7545"  
web3 = Web3(Web3.HTTPProvider(ganache_url))

# 检查连接
if not web3.is_connected():
    raise Exception("无法连接到 Ganache, 请确保 Ganache 正在运行并监听端口 7545。")

# 使用 Ganache 提供的账户和私钥
private_key = "0xb0ac7e4efd83a4c1628466d5863e6ea98acbb69e7ec45a151ae2bfe311ec9807"  
wallet_address = web3.eth.account.from_key(private_key).address

# ERC20 合约源码
contract_source_code = '''
pragma solidity ^0.8.0;

contract MyToken {
    string public name = "TRIGGERCOIN";
    string public symbol = "TGC";
    uint8 public decimals = 18;
    uint256 public totalSupply;
    
    mapping(address => uint256) public balanceOf;
    mapping(address => mapping(address => uint256)) public allowance;

    event Transfer(address indexed from, address indexed to, uint256 value);
    event Approval(address indexed owner, address indexed spender, uint256 value);

    constructor(uint256 initialSupply) {
        totalSupply = initialSupply * (10 ** uint256(decimals));
        balanceOf[msg.sender] = totalSupply;
    }

    function transfer(address to, uint256 value) public returns (bool success) {
        require(balanceOf[msg.sender] >= value, "Insufficient balance");
        balanceOf[msg.sender] -= value;
        balanceOf[to] += value;
        emit Transfer(msg.sender, to, value);
        return true;
    }

    function approve(address spender, uint256 value) public returns (bool success) {
        allowance[msg.sender][spender] = value;
        emit Approval(msg.sender, spender, value);
        return true;
    }

    function transferFrom(address from, address to, uint256 value) public returns (bool success) {
        require(balanceOf[from] >= value, "Insufficient balance");
        require(allowance[from][msg.sender] >= value, "Allowance exceeded");
        balanceOf[from] -= value;
        balanceOf[to] += value;
        allowance[from][msg.sender] -= value;
        emit Transfer(from, to, value);
        return true;
    }
}
'''

# 编译合约
compiled_sol = compile_source(contract_source_code, output_values=["abi", "bin"])
contract_id, contract_interface = compiled_sol.popitem()
abi = contract_interface['abi']
bytecode = contract_interface['bin']

# 部署合约
contract = web3.eth.contract(abi=abi, bytecode=bytecode)
nonce = web3.eth.get_transaction_count(wallet_address)

transaction = contract.constructor(1000000).build_transaction({
    'chainId': 1337,  # Ganache 默认 Chain ID
    'gas': 2000000,
    'gasPrice': web3.to_wei('10', 'gwei'),
    'nonce': nonce,
})

signed_tx = web3.eth.account.sign_transaction(transaction, private_key)
tx_hash = web3.eth.send_raw_transaction(signed_tx.raw_transaction)
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)

# 获取合约地址
contract_address = tx_receipt.contractAddress
print(f"合约已部署到地址: {contract_address}")


# 与部署的合约交互
deployed_contract = web3.eth.contract(address=contract_address, abi=abi)

def send_tokens(receiver_address, amount):
    # 获取当前 nonce
    nonce = web3.eth.get_transaction_count(wallet_address)
    
    # 构建转账交易
    transfer_tx = deployed_contract.functions.transfer(receiver_address, amount).build_transaction({
        'chainId': 1337,
        'gas': 2000000,
        'gasPrice': web3.to_wei('10', 'gwei'),
        'nonce': nonce,
        'from': wallet_address,  # 明确指定发送方地址
    })

    signed_transfer_tx = web3.eth.account.sign_transaction(transfer_tx, private_key)
    transfer_tx_hash = web3.eth.send_raw_transaction(signed_transfer_tx.raw_transaction)
    
    print(f"发送转账交易，交易哈希: {transfer_tx_hash.hex()}")
    
    # 等待交易被确认
    transfer_receipt = web3.eth.wait_for_transaction_receipt(transfer_tx_hash)
    
    if transfer_receipt.status == 1:
        print("转账成功")
    else:
        print("转账失败")

    return transfer_receipt

# 进行多次转账
for i in range(10):  # 进行10次转账
    receiver_address = "0x32d39aa6A6560C7A1326374f43F4e8fd4489a490"
    amount = 1000  # 转账金额
    receipt = send_tokens(receiver_address, amount)
    
    # 验证余额
    sender_balance = deployed_contract.functions.balanceOf(wallet_address).call()
    receiver_balance = deployed_contract.functions.balanceOf(receiver_address).call()
    print(f"第 {i+1} 次转账后 - 发送者余额: {sender_balance} TGC, 接收者余额: {receiver_balance} TGC")

合约已部署到地址: 0x06a99DeDD5bc7ba8c94189062338264b743E33cE
发送转账交易，交易哈希: 060f5dfa26aa9cf1a6181c9f59c0e638dcf881517eebccd94894c558840ade5c
转账成功
第 1 次转账后 - 发送者余额: 999999999999999999999000 TGC, 接收者余额: 1000 TGC
发送转账交易，交易哈希: 3c1d3c6cd9fc6db820e201e398a0ddc13a1a79e11b1bcc94dd678685133b5c61
转账成功
第 2 次转账后 - 发送者余额: 999999999999999999998000 TGC, 接收者余额: 2000 TGC
发送转账交易，交易哈希: 3ea03c3be7267d9d9804d9aeb787a2eeafc501c6319b31d6b790f1517837df41
转账成功
第 3 次转账后 - 发送者余额: 999999999999999999997000 TGC, 接收者余额: 3000 TGC
发送转账交易，交易哈希: bec1156fca8c1405d7522b9423c04bf467936258c9e25306249b8780b7f1a3aa
转账成功
第 4 次转账后 - 发送者余额: 999999999999999999996000 TGC, 接收者余额: 4000 TGC
发送转账交易，交易哈希: f019f8990e2701d1a58eaa11dc358b044ef78a73b1df16f07703b9b8e0d33674
转账成功
第 5 次转账后 - 发送者余额: 999999999999999999995000 TGC, 接收者余额: 5000 TGC
发送转账交易，交易哈希: a99bd729e24155536ae7b1d8d955d6ec00b6c1e7b42d2a5fe11ede015c5fee78
转账成功
第 6 次转账后 - 发送者余额: 999999999999999999994000 TGC, 接收者余额: 6000 TGC
发送转账交易，交易哈希: 7c47acef0b374fbed94c6a176bc2056a349f94f68fb58b3292be

In [5]:
from solcx import install_solc, set_solc_version

# 安装Solidity 0.8.0
install_solc('0.8.0')

# 设置为当前使用的版本（如果需要）
set_solc_version('0.8.0')

# 现在您可以尝试重新编译您的合约